In [1]:
import numpy as np
# load the split datasets from the files

X_train = np.load("/Feature_extraction_wav2vec/data_split/train_features.npy")
y_train = np.load("/Feature_extraction_wav2vec/data_split/train_labels.npy")
X_val = np.load("/Feature_extraction_wav2vec/data_split/val_features.npy")
y_val = np.load("/Feature_extraction_wav2vec/data_split/val_labels.npy")
X_test = np.load("/Feature_extraction_wav2vec/data_split/test_features.npy")
y_test = np.load("/Feature_extraction_wav2vec/data_split/test_labels.npy")


# Build a BILSMT model

In [3]:
from keras.layers.serialization import activation
import keras as keras
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense, SpatialDropout1D,Dropout
early_stop = EarlyStopping(monitor='val_loss', patience=6)
input_shape = (X_train.shape[1], X_train.shape[2])

model = Sequential()

#using 1layer BILSMT 
model.add(Bidirectional(LSTM(units=128,kernel_regularizer=keras.regularizers.l2(0.01))
                                    ,merge_mode='sum' ,input_shape=input_shape))
model.add(Dropout(0.2))


# output layer softmax
model.add(keras.layers.Dense(6, activation='softmax'))
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 128)              918528    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 919,302
Trainable params: 919,302
Non-trainable params: 0
_________________________________________________________________


In [3]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=40, callbacks=[early_stop])

Epoch 1/40
  6/293 [..............................] - ETA: 10:12 - loss: 1.8221 - accuracy: 0.1771

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
from keras.models import load_model
model.save("/Feature_extraction_wav2vec/Test_Models/large_dataset/modelBILSTM.h5")

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2)

# create accuracy sublpot
axs[0].plot(history.history["accuracy"], label="train accuracy")
axs[0].plot(history.history["val_accuracy"], label="test accuracy")
axs[0].set_ylabel("Accuracy")
axs[0].legend(loc="lower right")
axs[0].set_title("Accuracy eval")

# create error sublpot
axs[1].plot(history.history["loss"], label="train error")
axs[1].plot(history.history["val_loss"], label="test error")
axs[1].set_ylabel("Error")
axs[1].set_xlabel("Epoch")
axs[1].legend(loc="upper right")
axs[1].set_title("Error eval")

plt.show()


In [ ]:
def predict(model, X, y):
    """Predict a single sample using the trained model
    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))

In [ ]:
X_to_predict = X_test[41]
y_to_predict = y_test[41]

# predict sample
predict(model, X_to_predict, y_to_predict)


In [ ]:
predictions = []

# loop through the test set
for i in range(len(X_test)):
    # select the current sample from X_test and y_test
    X_to_predict = X_test[i]
    y_to_predict = y_test[i]
    
    # make the prediction using the model
    pred = predict(model, X_to_predict, y_to_predict)
    
    # append the prediction to the list of predictions
    predictions.append(pred)

In [5]:
labels_list = ['0','0','1','1','1','2','2','3']
labelss = [eval(i) for i in labels_list]
print(labelss)

[0, 0, 1, 1, 1, 2, 2, 3]
